In [1]:
# Step 1) import required libraries
import tkinter as tk
from tkinter import ttk
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
from PIL import Image, ImageTk
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from IPython.core.getipython import get_ipython
from serial.tools.list_ports import comports
import threading
import serial
from datetime import datetime
import time
import pytz
import statistics
import pandas as pd
import pymongo
from PIL import Image, ImageTk

client = pymongo.MongoClient("mongodb://localhost:27017/")  # Update the connection string with your MongoDB URI
database_name = "SpicerNew"  # Update with your desired database name
collection_name = "GaugeData"  # Update with your desired collection name
db = client[database_name]
collection = db[collection_name]

#Step 2) Define the SPC rules classs to check the rules
class SPCRules:
    def __init__(self, mean, std_dev, consecutive_count=0):
        self.mean = mean
        self.std_dev = std_dev

        # RULE 2 variables
        self.consecutive_count = consecutive_count
        self.prev_side = None

        # RULE 3 variables
        self.prev_data_point = None  # Initialize to None
        self.steady_count = 0  # Initialize to 0

        # RULE 4 variables
        self.prev_data_point2 = None
        self.trend = None  # Initialize trend to None
        self.consecutive_count2 = 0

        # Rule 5 variables
        self.consecutive_count5 = 0
        self.prev_points = []

        # Rule 6 variables
        self.consecutive_count6 = 0
        self.prev_points2 = []
        self.flag_count = 0

        # Rule 7 variable
        self.consecutive_count7 = 0

        # Rule 8 variable
        self.consecutive_count3 = 0
    def setfun(self,mean,std_dev,consecutive_count=0):
        self.mean = mean
        self.std_dev = std_dev

        # RULE 2 variables
        self.consecutive_count = consecutive_count
        self.prev_side = None

        # RULE 3 variables
        self.prev_data_point = None  # Initialize to None
        self.steady_count = 0  # Initialize to 0

        # RULE 4 variables
        self.prev_data_point2 = None
        self.trend = None  # Initialize trend to None
        self.consecutive_count2 = 0

        # Rule 5 variables
        self.consecutive_count5 = 0
        self.prev_points = []

        # Rule 6 variables
        self.consecutive_count6 = 0
        self.prev_points2 = []
        self.flag_count = 0

        # Rule 7 variable
        self.consecutive_count7 = 0

        # Rule 8 variable
        self.consecutive_count3 = 0

    def rule_1(self, data_point):
        """
        Rule 1: Any single data point falls outside 3-sigma control limits.
        """
        upper_limit = self.mean + 3 * self.std_dev
        lower_limit = self.mean - 3 * self.std_dev
        print(self.mean)
        if data_point > upper_limit:
            return 0
        if data_point < lower_limit:
            return 0
        return 1

    def rule_2(self, data_point):
        """
        Rule 2: Eight consecutive data points on both sides of the mean.Then 9th is invalid
        """
        if data_point > self.mean:
            side = 'above'
        elif data_point < self.mean:
            side = 'below'
        else:
            side = None

        if self.prev_side is None:
            # Start with the current side
            self.prev_side = side
            self.consecutive_count = 1
        elif side == self.prev_side:
            # Same side as the previous data point
            self.consecutive_count += 1
            if self.consecutive_count > 8:
                return 0  # Violation of Rule 8
        else:
            # Different side from the previous data point
            self.prev_side = side
            self.consecutive_count = 1

        return 1  # Within the limits

    def rule_3(self, data_point):
        """
        Rule 3: Six consecutive data points steadily increasing or decreasing.
        """
        if self.prev_data_point is None:
            self.prev_data_point = data_point
            self.steady_count = 1
            return 1

        if data_point > self.prev_data_point:
            if self.steady_count >= 0:
                self.steady_count += 1
            else:
                self.steady_count = 1
        elif data_point < self.prev_data_point:
            if self.steady_count <= 0:
                self.steady_count -= 1
            elif self.steady_count == 1:
                self.steady_count = -2  # For assuming the fact that when the initial data point was lead by 1 in increasing case
            else:
                self.steady_count = -1
        else:
            self.steady_count = 0

        if abs(self.steady_count) == 6:
            # Six consecutive data points steadily increasing or decreasing
            self.prev_data_point = None
            return 0  # Invalid
        else:
            self.prev_data_point = data_point
            return 1  # Valid

    def rule_4(self, data_point):
        """
        Rule 4: Fourteen consecutive data points alternating up and down.
        """
        if self.prev_data_point2 is None:
            # Initial value for the new data point
            self.prev_data_point2 = data_point
            self.trend = None
            self.consecutive_count2 = 1

        current_trend = None  # Initialize current_trend as None
        if data_point > self.prev_data_point2:
            current_trend = "up"
        elif data_point < self.prev_data_point2:
            current_trend = "down"

        if current_trend != self.trend:
            self.trend = current_trend
            self.consecutive_count2 += 1

        if self.consecutive_count2 == 14:
            self.prev_data_point2 = None
            return 0
        else:
            self.prev_data_point2 = data_point
            return 1

    def rule_5(self, data_point):
        """
        Rule 5: 2 out 3 points in a row are more than 2 standard deviations from mean in the same direction
        """
        if data_point > self.mean + (2 * self.std_dev):
            self.consecutive_count5 += 1
        elif data_point < self.mean - (2 * self.std_dev):
            self.consecutive_count5 -= 1
        else:
            self.consecutive_count5 = 0
        if abs(self.consecutive_count5) == 2:
            self.prev_points = []
            return 0
        self.prev_points.append(data_point)
        if len(self.prev_points) == 3:
            if self.prev_points[0] > self.mean + (2 * self.std_dev) and self.prev_points[2] > self.mean + (2 * self.std_dev):
                return 0
            elif self.prev_points[0] < self.mean - (2 * self.std_dev) and self.prev_points[2] < self.mean - (2 * self.std_dev):
                return 0
        return 1

    def rule_6(self, data_point):
        """
        Rule 6: Four out of five in a row are more than 1 standard deviation from the mean in the same direction.
        then the fourth point will be invalid
        """
        if data_point > self.mean + self.std_dev:
            self.consecutive_count6 += 1
        elif data_point < self.mean - self.std_dev:
            self.consecutive_count6 -= 1
        else:
            self.consecutive_count6 = 0
        if abs(self.consecutive_count6) == 4:
            self.prev_points2 = []
            return 0
        self.prev_points2.append(data_point)
        if len(self.prev_points2) == 5:
            self.flag_count = 0
            for i in range(len(self.prev_points2)):
                if self.prev_points2[i] > self.mean + self.std_dev:
                    self.flag_count += 1
                elif self.prev_points2[i] < self.mean - self.std_dev:
                    self.flag_count -= 1
            if abs(self.flag_count) == 4:
                return 0
        return 1

    def rule_7(self, data_point):
        """
        Rule 7: Fifteen consecutive data points within 1-sigma limits on either side of the mean.
        """
        if self.mean - self.std_dev <= data_point <= self.mean + self.std_dev:
            self.consecutive_count7 += 1
            if self.consecutive_count7 >= 15:
                return 0
        else:
            self.consecutive_count7 = 0
        return 1

    def rule_8(self, data_point):
        """
        Rule 8: Eight consecutive data points are out of 1 standard deviation above or below, then the 8th point will be invalid
        """
        if data_point > self.mean + self.std_dev or data_point < self.mean - self.std_dev:
            self.consecutive_count3 += 1
            if self.consecutive_count3 >= 8:
                return 0
        else:
            self.consecutive_count3 = 0
        return 1


    
    
excel_file = 'GAUGE DATA SHEET.xlsx'
df1 = pd.read_excel(excel_file)
csv_file = 'GAUGEDATA.csv'
df1.to_csv(csv_file, index=False)

# Step 3)  Create the main application window
root = tk.Tk()
root.title("Serial Data Analysis")


# Step 4) Initialize The required variables
ser = None
data_point_count = 1
data_point_count1 = 1
x, y = [], []
x1,y1=[],[]
max_data_points = 15
mean_value =0
std_deviation = 0
mean_value1=0
std_deviation1 = 0
SPCrules = SPCRules(mean_value, std_deviation)
SPCrules1 = SPCRules(mean_value1,std_deviation1)
com_port = None
graph=0


# Step 6) Define required function
 
 #Stpe 6.1) Define funtion to set com port and baud rate
def abort():
    root.destroy()
    
    
def detect_com_ports():
    # Use list_ports to detect available COM ports
    ports = [port.device for port in comports()]
    return ports

def open_serial_port():
    global ser,com_port
    available_ports = detect_com_ports()
    if not available_ports:
        connection_status_label.config(text="No COM ports detected")
        return
    selected_port = available_ports[1]
    com_port=selected_port
    print(f"Selected COM Port: {selected_port}")

    # Update the COM port entry with the detected port
    com_port_entry.delete(0, tk.END)
    com_port_entry.insert(0, selected_port)
    baud_rate = baud_rate_entry.get()
    try:
        ser = serial.Serial(com_port, baud_rate, timeout=1)
        connection_status_label.config(text=f"Connected to {com_port} at {baud_rate} baud")
        print(f"Connected to {com_port} at {baud_rate} baud")
    except Exception as e:
        connection_status_label.config(text="could not open port")
        print(f"Error: {e}")
    
    
 #Step 6.2) Define function to set mean and standard deviation
def apply_manual_values():
    global mean_value, std_deviation, graphflag,mean_value1,std_deviation1 
    try:
        mean_value = float(mean_entry.get())
        std_deviation = float(std_dev_entry.get())
        SPCrules.setfun(mean_value, std_deviation)
        
        mean_value1 = float(mean1_entry.get())
        std_deviation1 = float(std_dev1_entry.get())
        SPCrules1.setfun(mean_value1, std_deviation1)
        connection_status_label2.config(text="Mean and standard deviation applied")
        
         #create an object for this class
         # Start a thread to read and update the graph
        data_thread = threading.Thread(target=read_and_update_graph)
        ax.legend(loc='upper left')
        data_thread.daemon = True
        data_thread.start()
    except ValueError:
        connection_status_label2.config(text="Invalid Input")
        print("Invalid input. Please enter valid numeric values for mean and standard deviation.")

        
def read_and_update_graph():
    global data_point_count,data_point_count1, mean_value, std_deviation,graph,mean_value1, std_deviation1
    while (True):
        if ser and (graph%2) == 0:
            #print("G1")
            #print(y) #checking the values
            data = ser.readline()
            input_string = data.decode('utf-8')
            local_timezone = pytz.timezone('Asia/Kolkata')
            timestamp = datetime.fromtimestamp(time.time(), tz=local_timezone).strftime('%Y-%m-%d %H:%M:%S')
            parts = input_string.split(',')
            if len(parts) >= 2:
                numeric_value = float(parts[0])
                result_dict = {'MachineName':mn,'PartNumber':selected_option.get(),'Time': timestamp, 'Data': numeric_value}
                collection.insert_one(result_dict)
                x.append(data_point_count)
                y.append(numeric_value)
                if len(x) > max_data_points:
                    x.pop(0)
                    y.pop(0)
                try:
                    df = pd.read_csv('data26.csv')
                except FileNotFoundError:
                    df = pd.DataFrame(columns=['Time', 'Data'])
                df = pd.concat([df, pd.DataFrame([result_dict])])
                df.to_csv('data26.csv', index=False)
                data_array = df['Data'].values
                rule_messages = []

                # Check for rule violations
                
                #Check for RULE 1
                if SPCrules.rule_1(numeric_value)==0:
                    rule_messages.append("Rule 1")
                    rule_status_label2.config(text="One point is more than 3 standard deviations from the mean")
                    rule_labels[0].config(bg="red")
                    
                    #offset calculation
                    #1 clear beffore updating values
                    offset_entry.delete(0, tk.END)
                    #2 calculate the offset and update
                    offset = numeric_value - (mean_value + std_deviation)
                    offset_entry.insert(0, str(offset))
                    
                    SPCrules.setfun(mean_value, std_deviation) #Reintialise all the count 
                    ax.plot(data_point_count, numeric_value, color='red', marker='o', markersize=5)
                #Check for RULE 2
                if SPCrules.rule_2(numeric_value)==0:
                    rule_messages.append("Rule 2")
                    rule_status_label2.config(text="Nine or more points in a row are on the same side of the mean")
                    rule_labels[1].config(bg="red")
                    
                    #offset calculation
                    #1 clear beffore updating values
                    offset_entry.delete(0, tk.END)
                    #2 calculate the offset and update
                    last_four_points = y[-4:]
                    avg_of_four_points = sum(last_four_points)/4
                    offset= avg_of_four_points - (mean_value + std_deviation)
                    offset_entry.insert(0, str(offset))
                    
                    SPCrules.setfun(mean_value, std_deviation) #Reintialise all the count 
                    ax.plot(data_point_count, numeric_value, color='red', marker='o', markersize=5)
                #Check for RULE 3
                if SPCrules.rule_3(numeric_value)==0:
                    rule_messages.append("Rule 3")
                    rule_status_label2.config(text="Six or more points in aroe are continually increasing or decreasing")
                    rule_labels[2].config(bg="red")
                    
                    #offset calculation
                    #1 clear beffore updating values
                    offset_entry.delete(0, tk.END)
                    #2 calculate the offset and update
                    last_four_points = y[-4:]
                    avg_of_four_points = sum(last_four_points)/4
                    offset= avg_of_four_points - (mean_value + std_deviation)
                    offset_entry.insert(0, str(offset))
                    
                    SPCrules.setfun(mean_value, std_deviation) #Reintialise all the count 
                    ax.plot(data_point_count, numeric_value, color='red', marker='o', markersize=5)
                #Check for RULE 4
                if SPCrules.rule_4(numeric_value)==0:
                    rule_messages.append("Rule 4")
                    rule_status_label2.config(text="Fourteen or more points in arow alternate in direction, increasing then decreasing")
                    rule_labels[3].config(bg="red")
                    
                    #offset calculation
                    #1 clear beffore updating values
                    offset_entry.delete(0, tk.END)
                    #2 calculate the offset and update
                    offset= numeric_value - mean_value
                    offset_entry.insert(0, str(offset))
                    
                    SPCrules.setfun(mean_value, std_deviation) #Reintialise all the count  
                    ax.plot(data_point_count, numeric_value, color='red', marker='o', markersize=5)
                #Check for RULE 5
                if SPCrules.rule_5(numeric_value)==0:
                    rule_messages.append("Rule 5")
                    rule_status_label2.config(text="Two(or three) out of three points in a row are more than two standard deviation from the mean in same direction")
                    rule_labels[4].config(bg="red")
                    
                    #offset calculation
                    #1 clear beffore updating values
                    offset_entry.delete(0, tk.END)
                    #2 calculate the offset and update
                    last_three_points = y[-3:]
                    avg_of_three_points = sum(last_three_points)/3
                    offset= avg_of_three_points - (mean_value + std_deviation)
                    offset_entry.insert(0, str(offset))
                    
                    SPCrules.setfun(mean_value, std_deviation) #Reintialise all the count 
                    ax.plot(data_point_count, numeric_value, color='red', marker='o', markersize=5)
                #Check for RULE 6
                if SPCrules.rule_6(numeric_value)==0:
                    rule_messages.append("Rule 6")
                    rule_status_label2.config(text="Four(or five) out of five points in arow are more than one standard deviation from the mean in the same direction")
                    rule_labels[5].config(bg="red")
                    
                    #offset calculation
                    #1 clear beffore updating values
                    offset_entry.delete(0, tk.END)
                    #2 calculate the offset and update
                    last_five_points = y[-5:]
                    avg_of_five_points = sum(last_five_points)/5
                    offset= avg_of_five_points - (mean_value + std_deviation)
                    offset_entry.insert(0, str(offset))
                    
                    SPCrules.setfun(mean_value, std_deviation) #Reintialise all the count 
                    ax.plot(data_point_count, numeric_value, color='red', marker='o', markersize=5)
                #Check for RULE 7
                if SPCrules.rule_7(numeric_value)==0:
                    rule_messages.append("Rule 7")
                    rule_status_label2.config(text="Fifteen points in a row are all within one standard deviation of mean on either side of the mean")
                    rule_labels[6].config(bg="red")
                    
                    #offset calculation
                    #1 clear beffore updating values
                    offset_entry.delete(0, tk.END)
                    #2 calculate the offset and update
                    last_five_points = y[-5:]
                    avg_of_five_points = sum(last_five_points)/5
                    offset= avg_of_five_points - (mean_value + std_deviation)
                    offset_entry.insert(0, str(offset))
                    
                    SPCrules.setfun(mean_value, std_deviation) #Reintialise all the count 
                    ax.plot(data_point_count, numeric_value, color='red', marker='o', markersize=5)
                #Check for RULE 8
                if SPCrules.rule_8(numeric_value)==0:
                    rule_messages.append("Rule 8")
                    rule_status_label2.config(text="Eight points in a row exist with none within 1 standard deviation of the mean and the points are in both directions from the mean")
                    rule_labels[7].config(bg="red")
                    
                    #offset calculation
                    #1 clear beffore updating values
                    offset_entry.delete(0, tk.END)
                    #2 calculate the offset and update
                    last_four_points = y[-4:]
                    avg_of_four_points = sum(last_four_points)/4
                    offset= avg_of_four_points - (mean_value + std_deviation)
                    offset_entry.insert(0, str(offset))
                    
                    SPCrules.setfun(mean_value, std_deviation) #Reintialise all the count 
                    ax.plot(data_point_count, numeric_value, color='red', marker='o', markersize=5)
                
               #Plot Mean and Std _dev for given input....!!
                print("G1 ka mean "+str(mean_value))
                ax.cla()
                ax.plot(x, y, color='blue', marker='o', linestyle='-', label='Data')
                ax.axhline(mean_value, color='k', linestyle='--', label='Mean')
                ax.axhline(mean_value + std_deviation, color='g', linestyle='--', label='1 Sigma')
                ax.axhline(mean_value + 2 * std_deviation, color='y', linestyle='--', label='2 Sigma')
                ax.axhline(mean_value + 3 * std_deviation, color='r', linestyle='--', label='3 Sigma')
                ax.axhline(mean_value - std_deviation, color='g', linestyle='--')
                ax.axhline(mean_value - 2 * std_deviation, color='y', linestyle='--')
                ax.axhline(mean_value - 3 * std_deviation, color='r', linestyle='--')
                ax.legend(loc='upper left')
               
                for idx, message in enumerate(rule_messages):
                           ax.annotate(message, (data_point_count, numeric_value), textcoords="offset points", xytext=(0, 10 * (idx + 1)), ha='center')
                           ax.plot(data_point_count, numeric_value, color='red', marker='o', markersize=5)

        
                fig.canvas.draw()
                data_point_count += 1
                graph=graph+1
        elif ser and (graph%2) == 1:
            #print("G2")
            #print(y1) #checking the values
            data = ser.readline()
            input_string = data.decode('utf-8')
            local_timezone = pytz.timezone('Asia/Kolkata')
            timestamp = datetime.fromtimestamp(time.time(), tz=local_timezone).strftime('%Y-%m-%d %H:%M:%S')
            parts = input_string.split(',')
            if len(parts) >= 2:
                numeric_value = float(parts[0])
                result_dict = {'MachineName':mn,'PartNumber':selected_option.get(),'Time': timestamp, 'Data': numeric_value}
                collection.insert_one(result_dict)
                x1.append(data_point_count1)
                y1.append(numeric_value)
                if len(x) > max_data_points:
                    x1.pop(0)
                    y1.pop(0)
                try:
                    df = pd.read_csv('data26.csv')
                except FileNotFoundError:
                    df = pd.DataFrame(columns=['Time', 'Data'])
                    df = pd.concat([df, pd.DataFrame([result_dict])])
                    df.to_csv('data26.csv', index=False)
                    data_array = df['Data'].values
                    rule_messages = []

                # Check for rule violations
                
                #Check for RULE 1
                if SPCrules1.rule_1(numeric_value)==0:
                    rule_messages.append("Rule 1")
                    rule_status_label2.config(text="One point is more than 3 standard deviations from the mean")
                    rule_labels1[0].config(bg="red")
                    
                    #offset calculation
                    #1 clear beffore updating values
                    offset1_entry.delete(0, tk.END)
                    #2 calculate the offset and update
                    offset1 = numeric_value - (mean_value1 + std_deviation1)
                    offset1_entry.insert(0, str(offset1))
                    
                    SPCrules1.setfun(mean_value1, std_deviation1) #Reintialise all the count 
                    ax1.plot(data_point_count1, numeric_value, color='red', marker='o', markersize=5)
                #Check for RULE 2
                if SPCrules1.rule_2(numeric_value)==0:
                    rule_messages.append("Rule 2")
                    rule_status_label2.config(text="Nine or more points in a row are on the same side of the mean")
                    rule_labels1[1].config(bg="red")
                    
                    #offset calculation
                    #1 clear beffore updating values
                    offset1_entry.delete(0, tk.END)
                    #2 calculate the offset and update
                    last_four_points = y1[-4:]
                    avg_of_four_points = sum(last_four_points)/4
                    offset1= avg_of_four_points - (mean_value1 + std_deviation1)
                    offset1_entry.insert(0, str(offset1))
                    
                    SPCrules1.setfun(mean_value1, std_deviation1) #Reintialise all the count 
                    ax1.plot(data_point_count1, numeric_value, color='red', marker='o', markersize=5)
                #Check for RULE 3
                if SPCrules1.rule_3(numeric_value)==0:
                    rule_messages.append("Rule 3")
                    rule_status_label2.config(text="Six or more points in aroe are continually increasing or decreasing")
                    rule_labels1[2].config(bg="red")
                    
                    #offset calculation
                    #1 clear beffore updating values
                    offset1_entry.delete(0, tk.END)
                    #2 calculate the offset and update
                    last_four_points = y1[-4:]
                    avg_of_four_points = sum(last_four_points)/4
                    offset1= avg_of_four_points - (mean_value1 + std_deviation1)
                    offset1_entry.insert(0, str(offset1))
                    
                    SPCrules1.setfun(mean_value1, std_deviation1) #Reintialise all the count 
                    ax1.plot(data_point_count1, numeric_value, color='red', marker='o', markersize=5)
                #Check for RULE 4
                if SPCrules1.rule_4(numeric_value)==0:
                    rule_messages.append("Rule 4")
                    rule_status_label2.config(text="Fourteen or more points in arow alternate in direction, increasing then decreasing")
                    rule_labels1[3].config(bg="red")
                    
                    #offset calculation
                    #1 clear beffore updating values
                    offset1_entry.delete(0, tk.END)
                    #2 calculate the offset and update
                    offset1= numeric_value - mean_value1
                    offset1_entry.insert(0, str(offset1))
                    
                    SPCrules1.setfun(mean_value1, std_deviation1) #Reintialise all the count  
                    ax1.plot(data_point_count1, numeric_value, color='red', marker='o', markersize=5)
                #Check for RULE 5
                if SPCrules.rule_5(numeric_value)==0:
                    rule_messages.append("Rule 5")
                    rule_status_label2.config(text="Two(or three) out of three points in a row are more than two standard deviation from the mean in same direction")
                    rule_labels1[4].config(bg="red")
                    
                    #offset calculation
                    #1 clear beffore updating values
                    offset1_entry.delete(0, tk.END)
                    #2 calculate the offset and update
                    last_three_points = y1[-3:]
                    avg_of_three_points = sum(last_three_points)/3
                    offset1= avg_of_three_points - (mean_value1 + std_deviation1)
                    offset1_entry.insert(0, str(offset1))
                    
                    SPCrules1.setfun(mean_value1, std_deviation1) #Reintialise all the count 
                    ax1.plot(data_point_count1, numeric_value, color='red', marker='o', markersize=5)
                #Check for RULE 6
                if SPCrules.rule_6(numeric_value)==0:
                    rule_messages.append("Rule 6")
                    rule_status_label2.config(text="Four(or five) out of five points in arow are more than one standard deviation from the mean in the same direction")
                    rule_labels1[5].config(bg="red")
                    
                    #offset calculation
                    #1 clear beffore updating values
                    offset1_entry.delete(0, tk.END)
                    #2 calculate the offset and update
                    last_five_points = y1[-5:]
                    avg_of_five_points = sum(last_five_points)/5
                    offset1= avg_of_five_points - (mean_value1 + std_deviation1)
                    offset1_entry.insert(0, str(offset1))
                    
                    SPCrules1.setfun(mean_value1, std_deviation1) #Reintialise all the count 
                    ax1.plot(data_point_count1, numeric_value, color='red', marker='o', markersize=5)
                #Check for RULE 7
                if SPCrules.rule_7(numeric_value)==0:
                    rule_messages.append("Rule 7")
                    rule_status_label2.config(text="Fifteen points in a row are all within one standard deviation of mean on either side of the mean")
                    rule_labels1[6].config(bg="red")
                    
                    #offset calculation
                    #1 clear beffore updating values
                    offset1_entry.delete(0, tk.END)
                    #2 calculate the offset and update
                    last_five_points = y1[-5:]
                    avg_of_five_points = sum(last_five_points)/5
                    offset1= avg_of_five_points - (mean_value1 + std_deviation1)
                    offset1_entry.insert(0, str(offset1))
                    
                    SPCrules1.setfun(mean_value1, std_deviation1) #Reintialise all the count 
                    ax1.plot(data_point_count1, numeric_value, color='red', marker='o', markersize=5)
                #Check for RULE 8
                if SPCrules.rule_8(numeric_value)==0:
                    rule_messages.append("Rule 8")
                    rule_status_label2.config(text="Eight points in a row exist with none within 1 standard deviation of the mean and the points are in both directions from the mean")
                    rule_labels1[7].config(bg="red")
                    
                    #offset calculation
                    #1 clear beffore updating values
                    offset1_entry.delete(0, tk.END)
                    #2 calculate the offset and update
                    last_four_points = y1[-4:]
                    avg_of_four_points = sum(last_four_points)/4
                    offset1= avg_of_four_points - (mean_value1 + std_deviation1)
                    offset1_entry.insert(0, str(offset1))
                    
                    SPCrules1.setfun(mean_value1, std_deviation1) #Reintialise all the count 
                    ax1.plot(data_point_count1, numeric_value, color='red', marker='o', markersize=5)
                
               #Plot Mean and Std _dev for given input....!!
                print("G2 ka mean "+ str(mean_value1))
                ax1.cla()
                ax1.plot(x, y, color='blue', marker='o', linestyle='-', label='Data')
                ax1.axhline(mean_value1, color='k', linestyle='--', label='Mean')
                ax1.axhline(mean_value1 + std_deviation1, color='g', linestyle='--', label='1 Sigma')
                ax1.axhline(mean_value1 + 2 * std_deviation1, color='y', linestyle='--', label='2 Sigma')
                ax1.axhline(mean_value1 + 3 * std_deviation1, color='r', linestyle='--', label='3 Sigma')
                ax1.axhline(mean_value1 - std_deviation1, color='g', linestyle='--')
                ax1.axhline(mean_value1 - 2 * std_deviation1, color='y', linestyle='--')
                ax1.axhline(mean_value1 - 3 * std_deviation1, color='r', linestyle='--')
                ax1.legend(loc='upper left')
               
                for idx, message in enumerate(rule_messages):
                           ax1.annotate(message, (data_point_count1, numeric_value), textcoords="offset points", xytext=(0, 10 * (idx + 1)), ha='center')
                           ax1.plot(data_point_count1, numeric_value, color='red', marker='o', markersize=5)

        
                fig1.canvas.draw()
                data_point_count1 += 1
                graph=graph+1
                

                
#step 10)
flag = 0
mn=""
def update_label():
    global mn
    flag = 1
    mean_entry.delete(0, tk.END)
    std_dev_entry.delete(0, tk.END)
    mean1_entry.delete(0, tk.END)
    std_dev1_entry.delete(0, tk.END)
    machine_entry.delete(0, tk.END)
    
    drop_down_label.config(text="Selected Option: " + selected_option.get())
    df1['PART NUMBER '] = df1['PART NUMBER '].astype(str)
    condition = (df1['PART NUMBER '] == selected_option.get())
    
    selected_option_mean = df1.loc[condition, 'MEAN ']
    selected_option_std_dev = df1.loc[condition, 'Standard deviation']
    selected_option_machine = df1.loc[condition, 'Machine name']
    selected_option_mean1 = df1.loc[condition, 'Mean2']
    selected_option_std_dev1 = df1.loc[condition, 'Std_dev2']
    
    if not selected_option_mean.empty:
        sm = selected_option_mean.iloc[0]  # Get the first value if it exists
        sd = selected_option_std_dev.iloc[0]  # Get the first value if it exists
        mn = selected_option_machine.iloc[0] 
        sm1= selected_option_mean1.iloc[0]
        sd1= selected_option_std_dev1.iloc[0] 
        mean_entry.insert(0, str(sm))
        std_dev_entry.insert(0, str(sd))
        mean1_entry.insert(0, str(sm1))
        std_dev1_entry.insert(0, str(sd1))
        machine_entry.insert(0, str(mn))
    else:
        # Handle the case where no match was found
        mean_entry.insert(0, "No Data")
        std_dev_entry.insert(0, "No Data")
        machine_entry.insert(0, "No Data")
        mean1_entry.insert(0, "No Data")
        std_dev1_entry.insert(0, "No Data")
        
#Step GUI designing in grid system:-

#set the background color
new_color ="#4682B4"
root.configure(bg=new_color)  # Change the background color of the main window
custom_font = ('Arial', 18)


# Configure column weights
for i in range(6):
    root.columnconfigure(i, weight=1)

# Configure row weights
for i in range(20):
    root.rowconfigure(i, weight=1)


#Insert the image
# Load the image
image = Image.open("LOGO.png")  # Replace with the path to your image file
photo = ImageTk.PhotoImage(image) 
# Create a label to display the image
image_label = tk.Label(root, image=photo)
image_label.grid(row = 0,column = 0,columnspan=2,padx=20, pady=20)


#Title of the project
# Create a label and configure its background color
label = tk.Label(root, text="DANA ANAND INDIA PVT LTD", bg=new_color,font=custom_font)
label.grid(row = 0,column =2,padx=20, pady=20)


#INtialise the sytle for the GUI
style = ttk.Style()
style.configure("TButton", padding=6, relief="flat", background="#34cceb", foreground="red")
style.map("TButton",
          background=[("active", "#005599")],
          relief=[("active", "raised")]
          )
style.configure("TLabel", background="#f0f0f0")


#first row elements
com_port_label = ttk.Label(root, text="COM Port:",width=15, anchor="center")
com_port_entry = ttk.Entry(root)
baud_rate_label = ttk.Label(root, text="Baud Rate:",width=17, anchor="center")
baud_rate_entry = ttk.Entry(root)
baud_rate_entry.insert(0, "19200")
open_button = ttk.Button(root, text="Open Port", command=open_serial_port, style="TButton")

com_port_label.grid(row = 1, column = 0 ,padx =4, pady =2)
com_port_entry.grid(row = 1, column = 1 ,padx =2, pady =2)
baud_rate_label.grid(row = 1,column = 2,padx =2, pady =2)
baud_rate_entry.grid(row = 1,column = 3,padx =2, pady =2)
open_button.grid(row = 1,column = 4,padx =14, pady =12)

# Create a label to display connection status
connection_status_label = tk.Label(root, text="Not Connected", width=30)
connection_status_label.grid(row=1, column=5, padx=10, pady=10)


#SECOND row elements
drop_down_label = tk.Label(root, text = "Select Part",width=14, anchor="center")
drop_down_label.grid(row = 2,column = 0,padx =2, pady =2)
#Desgin a drop down
selected_option = tk.StringVar()
selected_option.set("Choose Part")
options = [(part_number)for part_number in df1['PART NUMBER ']]
dropdown = tk.OptionMenu(root,selected_option,*options)
dropdown.grid(row = 2,column = 1,padx =2, pady =2)

updatebutton = tk.Button(root,text = "Update", command = update_label,width=11)
updatebutton.grid(row = 2,column = 2,padx =2, pady =2)

#THIRD row element
#Step 8) Design GUI for Mean, Std deviation and apply button
mean_label = ttk.Label(root, text="Mean:",width=15, anchor="center")
mean_entry = ttk.Entry(root)
std_dev_label = ttk.Label(root, text="Standard Deviation:")
std_dev_entry = ttk.Entry(root)
machine_label = ttk.Label(root, text="Machine Name",width=15, anchor="center")
machine_entry = ttk.Entry(root)
offset_label = ttk.Label(root, text="offset",width=15, anchor="center")
offset_entry = ttk.Entry(root)
offset1_label = ttk.Label(root, text="offset1",width=15, anchor="center")
offset1_entry = ttk.Entry(root)

mean1_label = ttk.Label(root, text="Mean:",width=15, anchor="center")
mean1_entry = ttk.Entry(root)
std_dev1_label = ttk.Label(root, text="Standard Deviation:")
std_dev1_entry = ttk.Entry(root)

mean_label.grid(row = 4,column = 0,padx =2, pady =2)
mean_entry.grid(row = 4,column = 1,padx =2, pady =2)
std_dev_label.grid(row = 5,column = 0,padx =2, pady =2)
std_dev_entry.grid(row = 5,column = 1,padx =2, pady =2)

mean1_label.grid(row = 4,column = 4,padx =2, pady =2)
mean1_entry.grid(row = 4,column = 5,padx =2, pady =2)
std_dev1_label.grid(row = 5,column = 4,padx =2, pady =2)
std_dev1_entry.grid(row = 5,column = 5,padx =2, pady =2)

machine_label.grid(row = 3,column = 0,padx =2, pady =2)
machine_entry.grid(row = 3,column = 1,padx =2, pady =2)

offset_label.grid(row = 3,column = 2,padx =2, pady =2)
offset_entry.grid(row = 3,column = 3,padx =2, pady =2)
offset1_label.grid(row = 4,column = 2,padx =2, pady =2)
offset1_entry.grid(row = 4,column = 3,padx =2, pady =2)

apply_button = ttk.Button(root, text="Apply 1", command=apply_manual_values, style="TButton",width=11)
apply_button.grid(row = 3,column = 4,padx =2, pady =2)


connection_status_label2 = tk.Label(root, text="Not applied", width=30)
connection_status_label2.grid(row=3, column=5, padx=10, pady=10)


#Fourth row design
rule_status_label2 = tk.Label(root, text="No Rule violated", width=90)
rule_status_label2.grid(row=6, column=0,columnspan=3, padx=10, pady=10)
abort_button = ttk.Button(root, text="Abort", command=abort, style="TButton", width=11)
abort_button.grid(row=5, column=3, padx=2, pady=2)


#5th row element graph
#Step 9)DEfine SPCrule object and intialise mean std_dev integarte chart to GUI  
plt.rcParams['animation.html'] = 'jshtml'
fig = Figure(figsize=(6,4))
ax = fig.add_subplot(111)
canvas = FigureCanvasTkAgg(fig, master=root)
canvas.get_tk_widget().grid(row = 7,column = 0,columnspan=3,rowspan=10,padx =2, pady =2)

plt.rcParams['animation.html'] = 'jshtml'
fig1 = Figure(figsize=(6,4))
ax1= fig1.add_subplot(111)
canvas1 = FigureCanvasTkAgg(fig1, master=root)
canvas1.get_tk_widget().grid(row = 7,column = 4,columnspan=3,rowspan=10,padx =2, pady =2)

#Rule label design
rule_labels = []
for rule_num in range(1, 9):
    label_text = f"Rule {rule_num}"
    rule_label = tk.Label(root, text=label_text, width=5, anchor="center")
    rule_label.grid(row= 6+rule_num, column=3, padx=2, pady=2)
    rule_label.config(bg="green")
    rule_labels.append(rule_label)
    

rule_labels1 = []
for rule_num in range(1, 9):
    label_text1 = f"Rule {rule_num}"
    rule_label1 = tk.Label(root, text=label_text1, width=5, anchor="center")
    rule_label1.grid(row= 6+rule_num, column=7, padx=2, pady=2)
    rule_label1.config(bg="green")
    rule_labels1.append(rule_label1)
    


root.mainloop()